tgb - 2/3/2020 - The goal of this notebook is to develop custom tensorflow layers to rescale inputs in order to facilitate generalization. We will consider three input rescalings in this notebook:  
1) From specific humidity to relative humidity  
2) From T to T-T_{NS}, NS=Near-surface  
3) From T to T-T_{MA}, MA=Approximate moist adiabat

# Imports

In [1]:
from cbrain.imports import *
from cbrain.data_generator import *
from cbrain.cam_constants import *
from cbrain.losses import *
from cbrain.utils import limit_mem
from cbrain.layers import *
from cbrain.data_generator import DataGenerator
import tensorflow as tf
import tensorflow.math as tfm
import tensorflow_probability as tfp
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import xarray as xr
import numpy as np
from cbrain.model_diagnostics import ModelDiagnostics
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.image as imag
import scipy.integrate as sin
import cartopy.crs as ccrs
import matplotlib.ticker as mticker
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import pickle

TRAINDIR = '/local/Tom.Beucler/SPCAM_PHYS/'
DATADIR = '/project/meteo/w2w/A6/S.Rasp/SP-CAM/fluxbypass_aqua/'
PREFIX = '8col009_01_'
%cd /filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM

# Otherwise tensorflow will use ALL your GPU RAM for no reason
limit_mem()

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog


W0203 22:23:58.709062 23005436864320 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:145: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0203 22:23:58.709892 23005436864320 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:148: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W0203 22:23:58.710371 23005436864320 deprecation_wrapper.py:119] From /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/utils.py:148: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



/filer/z-sv-pool12c/t/Tom.Beucler/SPCAM/CBRAIN-CAM


# Input Rescaling 1: Specific humidity to relative humidity

## Moist thermodynamics library

In [2]:
# Moist thermodynamics library adapted to tf
def eliq(T):
    a_liq = np.float32(np.array([-0.976195544e-15,-0.952447341e-13,\
                                 0.640689451e-10,\
                      0.206739458e-7,0.302950461e-5,0.264847430e-3,\
                      0.142986287e-1,0.443987641,6.11239921]));
    c_liq = np.float32(-80.0)
    T0 = np.float32(273.16)
    return np.float32(100.0)*tfm.polyval(a_liq,tfm.maximum(c_liq,T-T0))

def eice(T):
    a_ice = np.float32(np.array([0.252751365e-14,0.146898966e-11,0.385852041e-9,\
                      0.602588177e-7,0.615021634e-5,0.420895665e-3,\
                      0.188439774e-1,0.503160820,6.11147274]));
    c_ice = np.float32(np.array([273.15,185,-100,0.00763685,0.000151069,7.48215e-07]))
    T0 = np.float32(273.16)
    return tf.where(T>c_ice[0],eliq(T),\
                   tf.where(T<=c_ice[1],np.float32(100.0)*(c_ice[3]+tfm.maximum(c_ice[2],T-T0)*\
                   (c_ice[4]+tfm.maximum(c_ice[2],T-T0)*c_ice[5])),\
                           np.float32(100.0)*tfm.polyval(a_ice,T-T0)))

def esat(T):
    T0 = np.float32(273.16)
    T00 = np.float32(253.16)
    omtmp = (T-T00)/(T0-T00)
    omega = tfm.maximum(np.float32(0.0),tfm.minimum(np.float32(1.0),omtmp))

    return tf.where(T>T0,eliq(T),tf.where(T<T00,eice(T),(omega*eliq(T)+(1-omega)*eice(T))))

def qv(T,RH,P0,PS,hyam,hybm):
    
    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    T = tf.cast(T,tf.float32)
    RH = tf.cast(RH,tf.float32)
    p = tf.cast(p,tf.float32)
    
    return R*esat(T)*RH/(Rv*p)
    # DEBUG 1
    # return esat(T)
    
def RH(T,qv,P0,PS,hyam,hybm):
    R = np.float32(287.0)
    Rv = np.float32(461.0)
    p = P0 * hyam + PS[:, None] * hybm # Total pressure (Pa)
    
    T = tf.cast(T,tf.float32)
    qv = tf.cast(qv,tf.float32)
    p = tf.cast(p,tf.float32)
    
    return Rv*p*qv/(R*esat(T))

## Layers to convert from specific humidity to relative humidity

In [3]:
class QV2RH(Layer):
    def __init__(self, inp_subQ, inp_divQ, inp_subRH, inp_divRH, hyam, hybm, **kwargs):
        """
        Call using ([input])
        Assumes
        prior: [QBP, 
        TBP, PS, SOLIN, SHFLX, LHFLX]
        Returns
        post(erior): [RHBP,
        TBP, PS, SOLIN, SHFLX, LHFLX]
        Arguments:
        inp_subQ = Normalization based on input with specific humidity (subtraction constant)
        inp_divQ = Normalization based on input with specific humidity (division constant)
        int_subRH = Normalization based on input with relative humidity (subtraction constant)
        inp_divRH = Normalization based on input with relative humidity (division constant)
        hyam = Constant a for pressure based on mid-levels
        hybm = Constant b for pressure based on mid-levels
        """
        self.inp_subQ, self.inp_divQ, self.inp_subRH, self.inp_divRH, self.hyam, self.hybm = \
            np.array(inp_subQ), np.array(inp_divQ), np.array(inp_subRH), np.array(inp_divRH), \
        np.array(hyam), np.array(hybm)
        # Define variable indices here
        # Input
        self.QBP_idx = slice(0,30)
        self.TBP_idx = slice(30,60)
        self.PS_idx = 60
        self.SHFLX_idx = 62
        self.LHFLX_idx = 63

        super().__init__(**kwargs)

    def build(self, input_shape):
        super().build(input_shape)

    def get_config(self):
        config = {'inp_subQ': list(self.inp_subQ), 'inp_divQ': list(self.inp_divQ),
                  'inp_subRH': list(self.inp_subRH), 'inp_divRH': list(self.inp_divRH),
                  'hyam': list(self.hyam),'hybm': list(self.hybm)}
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))
    
    def call(self, arrs):
        prior = arrs
        
        Tprior = prior[:,self.TBP_idx]*self.inp_divQ[self.TBP_idx]+self.inp_subQ[self.TBP_idx]
        qvprior = prior[:,self.QBP_idx]*self.inp_divQ[self.QBP_idx]+self.inp_subQ[self.QBP_idx]
        PSprior = prior[:,self.PS_idx]*self.inp_divQ[self.PS_idx]+self.inp_subQ[self.PS_idx]
        RHprior = (RH(Tprior,qvprior,P0,PSprior,self.hyam,self.hybm)-\
                    self.inp_subRH[self.QBP_idx])/self.inp_divRH[self.QBP_idx]
        
        post = tf.concat([tf.cast(RHprior,tf.float32),prior[:,30:]], axis=1)
        
        return post

    def compute_output_shape(self,input_shape):
        """Input shape + 1"""
        return (input_shape[0][0])

# Pre-process data for the experiment

## Specific humidity, norm file for (-4K)

Here is the configuration file ```CI_SP_M4K_NORM.yml```:
```
# Climate-invariant - Specific humidity - Minus 4K - Norm
vars : [QBP, TBP, PS, SOLIN, SHFLX, LHFLX, PHQ, TPHYSTND, FSNT, FSNS, FLNT, FLNS]

in_dir : /project/meteo/w2w/A6/S.Rasp/SP-CAM/sp8fbp_minus4k
in_fns : sp8fbp_minus4k.cam2.h2.0001-01-0*-00000.nc

out_dir: /local/Tom.Beucler/SPCAM_PHYS/
out_fn: CI_SP_M4K_NORM_train.nc

val_in_fns: sp8fbp_minus4k.cam2.h2.0001-01-0*-00000.nc
val_out_fn: CI_SP_M4K_NORM_valid.nc

norm_fn: CI_SP_M4K_NORM_norm.nc
```

We pre-process it below:

In [6]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/CI_SP_M4K_NORM.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

## Specific humidity, training and validation for (-4K)

Here is the configuration file ```CI_SP_M4K.yml```:
```
# Climate-invariant - Specific humidity - Minus 4K - Training and validation
vars : [QBP, TBP, PS, SOLIN, SHFLX, LHFLX, PHQ, TPHYSTND, FSNT, FSNS, FLNT, FLNS]

in_dir : /project/meteo/w2w/A6/S.Rasp/SP-CAM/sp8fbp_minus4k
in_fns : sp8fbp_minus4k.cam2.h2.0001-*-0*-00000.nc

out_dir: /local/Tom.Beucler/SPCAM_PHYS/
out_fn: CI_SP_M4K_train.nc

val_in_fns: sp8fbp_minus4k.cam2.h2.0001-*-1*-00000.nc
val_out_fn: CI_SP_M4K_valid.nc
```

In [7]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/CI_SP_M4K.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

## Specific humidity, norm file for (+4K)

Here is the configuration file ```CI_SP_P4K_NORM.yml```:
```
# Climate-invariant - Specific humidity - Plus 4K - Norm
vars : [QBP, TBP, PS, SOLIN, SHFLX, LHFLX, PHQ, TPHYSTND, FSNT, FSNS, FLNT, FLNS]

in_dir : /project/meteo/w2w/A6/S.Rasp/SP-CAM/sp8fbp_4k
in_fns : sp8fbp_4k.cam2.h2.0001-01-0*-00000.nc

out_dir: /local/Tom.Beucler/SPCAM_PHYS/
out_fn: CI_SP_P4K_NORM_train.nc

val_in_fns: sp8fbp_4k.cam2.h2.0001-01-0*-00000.nc
val_out_fn: CI_SP_P4K_NORM_valid.nc

norm_fn: CI_SP_P4K_NORM_norm.nc
```

In [8]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/CI_SP_P4K_NORM.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

## Specific humidity, training and validation for (+4K)

Here is the configuration file ```CI_SP_P4K.yml ```:
```
# Climate-invariant - Specific humidity - Plus 4K - Training and validation
vars : [QBP, TBP, PS, SOLIN, SHFLX, LHFLX, PHQ, TPHYSTND, FSNT, FSNS, FLNT, FLNS]

in_dir : /project/meteo/w2w/A6/S.Rasp/SP-CAM/sp8fbp_4k
in_fns : sp8fbp_4k.cam2.h2.0001-*-0*-00000.nc

out_dir: /local/Tom.Beucler/SPCAM_PHYS/
out_fn: CI_SP_P4K_train.nc

val_in_fns: sp8fbp_4k.cam2.h2.0001-*-1*-00000.nc
val_out_fn: CI_SP_P4K_valid.nc
```

In [9]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/CI_SP_P4K.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

## Relative humidity, norm file for (-4K)

Here is the configuration file ```CI_RH_M4K_NORM.yml```:
```
# Climate-invariant - Relative humidity - Minus 4K - Norm
vars : [RH, TBP, PS, SOLIN, SHFLX, LHFLX, PHQ, TPHYSTND, FSNT, FSNS, FLNT, FLNS]

in_dir : /project/meteo/w2w/A6/S.Rasp/SP-CAM/sp8fbp_minus4k
in_fns : sp8fbp_minus4k.cam2.h2.0001-01-0*-00000.nc

out_dir: /local/Tom.Beucler/SPCAM_PHYS/
out_fn: CI_RH_M4K_NORM_train.nc

val_in_fns: sp8fbp_minus4k.cam2.h2.0001-01-0*-00000.nc
val_out_fn: CI_RH_M4K_NORM_valid.nc

norm_fn: CI_RH_M4K_NORM_norm.nc
```

In [10]:
!python preprocessing-11132019.py -c /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/CI_RH_M4K_NORM.yml

/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages

# Build and train models on (-4K) with and without the first input rescaling

## Generator using specific humidity inputs at (-4K)

In [4]:
scale_dict = load_pickle('./nn_config/scale_dicts/009_Wm2_scaling.pkl')

In [5]:
in_vars = ['QBP','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']

In [6]:
TRAINFILE = 'CI_SP_M4K_train_shuffle.nc'
NORMFILE = 'CI_SP_M4K_NORM_norm.nc'
VALIDFILE = 'CI_SP_M4K_valid.nc'

In [7]:
train_gen = DataGenerator(
    data_fn = TRAINDIR+TRAINFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

In [8]:
valid_gen = DataGenerator(
    data_fn = TRAINDIR+VALIDFILE,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    batch_size=1024,
    shuffle=True
)

Check that shapes of generated samples are consistent

In [9]:
print(train_gen[50][0].shape)
print(train_gen[50][1].shape)
print(valid_gen[78][0].shape)
print(valid_gen[78][1].shape)

(1024, 64)
(1024, 64)
(1024, 64)
(1024, 64)


## Generator using relative humidity inputs

In [10]:
scale_dict_RH = load_pickle('./nn_config/scale_dicts/009_Wm2_scaling.pkl')

In [11]:
scale_dict_RH['RH'] = 0.01*L_S/G, # Arbitrary 0.1 factor as specific humidity is generally below 2%  

In [12]:
in_vars_RH = ['RH','TBP','PS', 'SOLIN', 'SHFLX', 'LHFLX']
out_vars_RH = ['PHQ','TPHYSTND','FSNT', 'FSNS', 'FLNT', 'FLNS']

In [13]:
TRAINFILE_RH = 'CI_RH_M4K_NORM_train_shuffle.nc'
NORMFILE_RH = 'CI_RH_M4K_NORM_norm.nc'
VALIDFILE_RH = 'CI_RH_M4K_NORM_valid.nc'

In [14]:
train_gen_RH = DataGenerator(
    data_fn = TRAINDIR+TRAINFILE_RH,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = TRAINDIR+NORMFILE_RH,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict_RH,
    batch_size=1024,
    shuffle=True
)

Check that shapes of generated samples are consistent

In [15]:
print(train_gen_RH[50][0].shape)
print(train_gen_RH[50][1].shape)
print(train_gen_RH[78][0].shape)
print(train_gen_RH[78][1].shape)

(1024, 34)
(1024, 64)
(1024, 34)
(1024, 64)


## Build "brute-force" model

In [16]:
inp = Input(shape=(64,))
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
out = Dense(64, activation='linear')(densout)
Brute_force = tf.keras.models.Model(inp, out)

W0203 20:03:20.347390 23260197443392 deprecation.py:506] From /home/t/Tom.Beucler/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
Brute_force.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense (Dense)                (None, 128)               8320      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 128)               0     

## Build model converting specific humidity to relative humidity as inputs

In [18]:
inp = Input(shape=(64,))
inpRH = QV2RH(inp_subQ=train_gen.input_transform.sub, 
              inp_divQ=train_gen.input_transform.div, 
              inp_subRH=train_gen_RH.input_transform.sub, 
              inp_divRH=train_gen_RH.input_transform.div, 
              hyam=hyam, hybm=hybm)(inp)
densout = Dense(128, activation='linear')(inpRH)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
out = Dense(64, activation='linear')(densout)
Input_RH = tf.keras.models.Model(inp, out)

W0203 20:03:21.579846 23260197443392 deprecation.py:323] From <ipython-input-2-d8b861a31dd8>:20: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [19]:
Input_RH.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
q_v2rh (QV2RH)               (None, 64)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               8320      
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               1651

## Compile and train both models for Nep (e.g. 10) epochs

In [20]:
path_HDF5 = '/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/'
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_BF = ModelCheckpoint(path_HDF5+'CI01_BF.hdf5',save_best_only=True, monitor='val_loss', mode='min')
mcp_save_RH = ModelCheckpoint(path_HDF5+'CI01_RH.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [21]:
Brute_force.compile(tf.keras.optimizers.Adam(), loss=mse)
Input_RH.compile(tf.keras.optimizers.Adam(), loss=mse)

In [22]:
Nep = 10
Brute_force.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
              callbacks=[earlyStopping, mcp_save_BF])

Epoch 1/10
41376/41376 [==============================] - 252s 6ms/step - loss: 422.2687 - val_loss: 379.7717
Epoch 2/10
41376/41376 [==============================] - 251s 6ms/step - loss: 373.4242 - val_loss: 366.8192
Epoch 3/10
41376/41376 [==============================] - 252s 6ms/step - loss: 364.0181 - val_loss: 357.2235
Epoch 4/10
41376/41376 [==============================] - 250s 6ms/step - loss: 358.9917 - val_loss: 355.3069
Epoch 5/10
41376/41376 [==============================] - 251s 6ms/step - loss: 355.9355 - val_loss: 356.3034
Epoch 6/10
41376/41376 [==============================] - 250s 6ms/step - loss: 353.8153 - val_loss: 351.0591
Epoch 7/10
41376/41376 [==============================] - 251s 6ms/step - loss: 352.0802 - val_loss: 357.2676
Epoch 8/10
41376/41376 [==============================] - 252s 6ms/step - loss: 350.7423 - val_loss: 350.0558
Epoch 9/10
41376/41376 [==============================] - 251s 6ms/step - loss: 349.6989 - val_loss: 348.0157
Epoch 10/1

In [23]:
Nep = 10
Input_RH.fit_generator(train_gen, epochs=Nep, validation_data=valid_gen,\
              callbacks=[earlyStopping, mcp_save_RH])

Epoch 1/10
41376/41376 [==============================] - 302s 7ms/step - loss: 613.4270 - val_loss: 569.8912
Epoch 2/10
41376/41376 [==============================] - 301s 7ms/step - loss: 548.4543 - val_loss: 539.9586
Epoch 3/10
41376/41376 [==============================] - 298s 7ms/step - loss: 532.8021 - val_loss: 531.1984
Epoch 4/10
41376/41376 [==============================] - 298s 7ms/step - loss: 524.3498 - val_loss: 520.8333
Epoch 5/10
41376/41376 [==============================] - 298s 7ms/step - loss: 518.5004 - val_loss: 524.0867
Epoch 6/10
41376/41376 [==============================] - 301s 7ms/step - loss: 513.9757 - val_loss: 526.8451
Epoch 7/10
41376/41376 [==============================] - 300s 7ms/step - loss: 510.1706 - val_loss: 514.2112
Epoch 8/10
41376/41376 [==============================] - 300s 7ms/step - loss: 506.7798 - val_loss: 513.0064
Epoch 9/10
41376/41376 [==============================] - 300s 7ms/step - loss: 504.3581 - val_loss: 513.6531
Epoch 10/1

# Compare the models' abilities to generalize to (+4K)

## Define configuration files for post-processing the two neural networks

Note that the architecture of the NN in the configuration file is not used (if the configuration file was not used to train the network). We only use it for the convenience of building a model diagnostics object.

Here is the configuration file ```CI_SP_M4K_CONFIG.yml```:
```
# [Configuration file] Climate-invariant - Specific humidity - Minus 4K - Training and validation
exp_name: 000_example
data_dir: /local/Tom.Beucler/SPCAM_PHYS/
train_fn: CI_SP_M4K_train_shuffle.nc
valid_fn: CI_SP_M4K_valid.nc
norm_fn: CI_SP_M4K_NORM_norm.nc

inputs: [QBP,TBP,PS,SOLIN,SHFLX,LHFLX]
outputs: [PHQ,TPHYSTND,FSNT,FSNS,FLNT,FLNS]

input_sub: mean
input_div: maxrs
output_dict: /home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/nn_config/scale_dicts/009_Wm2_scaling.pkl

hidden_layers: [1, 1]
epochs: 1
conservation_layer: False
loss: mse
```

## Build model diagnostics objects

In [6]:
path_HDF5 = '/local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA/'
config_file = 'CI_SP_M4K_CONFIG.yml'
data_file = ['CI_SP_M4K_valid.nc','CI_SP_P4K_valid.nc']
NNarray = ['CI01_BF.hdf5','CI01_RH.hdf5']
NNname = ['Brute force','Input RH']
dict_lay = {'SurRadLayer':SurRadLayer,'MassConsLayer':MassConsLayer,'EntConsLayer':EntConsLayer,
            'QV2RH':QV2RH,'eliq':eliq,'eice':eice,'esat':esat,'qv':qv,'RH':RH}

In [7]:
NN = {}; md = {};
%cd $TRAINDIR/HDF5_DATA
for i,NNs in enumerate(NNarray):
    print('NN name is ',NNs)
    path = path_HDF5+NNs
    NN[NNs] = load_model(path,custom_objects=dict_lay)
    md[NNs] = {}
    for j,data in enumerate(data_file):
        print('data name is ',data)
        md[NNs][data[13:-3]] = ModelDiagnostics(NN[NNs],
                                                '/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/pp_config/'+config_file,
                                                '/local/Tom.Beucler/SPCAM_PHYS/'+data)

/scratch-local/Tom.Beucler/SPCAM_PHYS/HDF5_DATA
NN name is  CI01_BF.hdf5
data name is  CI_SP_M4K_valid.nc
data name is  CI_SP_P4K_valid.nc
NN name is  CI01_RH.hdf5


/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/model_diagnostics.py:25: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)
W0203 22:25:18.026977 23005436864320 deprecation.py:323] From <ipython-input-2-d8b861a31dd8>:20: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


data name is  CI_SP_M4K_valid.nc
data name is  CI_SP_P4K_valid.nc


/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/model_diagnostics.py:25: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)
/home/t/Tom.Beucler/SPCAM/CBRAIN-CAM/notebooks/tbeucler_devlog/cbrain/model_diagnostics.py:25: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(f)
